# 导入工具包

In [1]:
import cv2
import mediapipe as mp

#进度条库
from tqdm import tqdm
import time
import matplotlib.pyplot as plt

#使用ipython魔法方法，将绘制得到的图像直接嵌入在notebook单元格中
%matplotlib inline

# 导入手部关键点检测模型

In [3]:
#导入solution
mp_hands = mp.solutions.hands

#导入模型
hands = mp_hands.Hands(static_image_mode=False,       #是静态图片还是连续视频帧
                      max_num_hands=4,                #最多检测几只手
                      min_detection_confidence=0.5,   #置信度阈值
                      min_tracking_confidence=0.5,    #追踪阈值
                      )

#导入绘图函数
mpDraw = mp.solutions.drawing_utils

# 处理单帧的函数

In [5]:
# 处理帧函数
def process_frame(img):
    # 水平镜像翻转图像，使得图中的左右手与真实左右手相对应
    #参数，1：水平翻转，0：竖直翻转，-1：水平和竖直都翻转
    img = cv2.flip(img,1)
    # BGR转RGB
    img_RGB = cv2.cvtColor(img,cv2.COLOR_BGRA2RGB)

    #将RGB图像输入模型。获取预测结果
    results = hands.process(img_RGB)
    
    
    if results.multi_hand_landmarks:   #如果检测到手
    #遍历每一只检测出的手
        for hand_idx in range(len(results.multi_hand_landmarks)):
            hand_21 = results.multi_hand_landmarks[hand_idx]    #获取手的所有关键点坐标
            mpDraw.draw_landmarks(img,hand_21,connections = mp_hands.HAND_CONNECTIONS)
        
    return img

# 调用摄像头获取每帧（模板）

In [6]:
# 调用摄像头逐帧实时处理模板
#不需修改任何代码，只需修改process_frame函数即可

import cv2

#获取摄像头，传入0表示获取系统默认摄像头
cap = cv2.VideoCapture()

#打开cap
cap.open(0)

#无限循环，直到break被触发
while cap.isOpened():
    #获取画面
    success,frame = cap.read()
    if not success:
        print('Error')
        break
        
    ## !!! 帧处理函数
    frame = process_frame(frame)
    
    #展示处理后的三通道图像
    cv2.imshow('my_window',frame)
    
    if cv2.waitKey(1) in [ord('q'),27]:    #按键盘上的q或esc退出（在英文输入法下）
        break
        
#关闭摄像头
cap.release()

#关闭图像窗口
cv2.destroyAllWindows()